# **1164. Product Price at a Given Date**

**Table: Products**
``` console
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+
```
(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.
 

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:**
**Products table:**
``` console
+------------+-----------+-------------+
| product_id | new_price | change_date |
+------------+-----------+-------------+
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |
+------------+-----------+-------------+
```
**Output:**
``` console 
+------------+-------+
| product_id | price |
+------------+-------+
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |
+------------+-------+
```

**Solution 1:**

In [ ]:
SELECT DISTINCT p.product_id, COALESCE(a.new_price,10) AS price
FROM Products p
LEFT JOIN
(SELECT product_id, new_price,RANK() OVER(PARTITION BY product_id ORDER BY change_date DESC) as rnk
FROM Products WHERE change_date <= '2019-08-16')a
ON p.product_id = a.product_id AND a.rnk = 1

**Solution 2:**

In [ ]:
SELECT DISTINCT product_id, 10 AS price 
FROM Products 
WHERE product_id NOT IN(SELECT DISTINCT product_id 
                        FROM Products 
                        WHERE change_date <='2019-08-16' )
UNION 
SELECT product_id, new_price AS price 
FROM Products 
WHERE (product_id,change_date) IN (SELECT product_id , MAX(change_date) AS change_date 
                                    FROM Products 
                                    WHERE change_date <='2019-08-16' 
                                    GROUP BY product_id)

**Solution 3:**

In [ ]:
SELECT p.product_id, 
       COALESCE((
           SELECT new_price 
           FROM Products 
           WHERE product_id = p.product_id AND change_date <= '2019-08-16'
           ORDER BY change_date DESC 
           LIMIT 1
       ), 10) AS price
FROM (SELECT DISTINCT product_id FROM Products) p;

**Solution 4:**

In [ ]:
# Write your MySQL query statement below
WITH cte AS
(SELECT *, RANK() OVER (PARTITION BY product_id ORDER BY change_date DESC) AS r 
FROM Products
WHERE change_date<= '2019-08-16')

SELECT product_id, new_price AS price
FROM cte
WHERE r = 1
UNION
SELECT product_id, 10 AS price
FROM Products
WHERE product_id NOT IN (SELECT product_id FROM cte)